In [56]:
#Import requests library and API key
import requests
from config import weather_api_key

# Import Dependencies
import pandas as pd

### Import & edit necessary DataFrames for API call

In [57]:
# Import DataFrame from Segment One
initial_2019 = pd.read_csv('../../Database/Data/jan_19_clean.csv')
initial_2020 = pd.read_csv('../../Database/Data/jan_20_clean.csv')
initial_2019

,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,1,2,20363,9E,N8688C,3280,11953,1195302,GNV,10397,1039707,ATL,601.0,0.0,0600-0659,722.0,0.0,0,0,300
1,1,2,20363,9E,N348PQ,3281,13487,1348702,MSP,11193,1119302,CVG,1359.0,0.0,1400-1459,1633.0,0.0,0,0,596
2,1,2,20363,9E,N8896A,3282,11433,1143302,DTW,11193,1119302,CVG,1215.0,0.0,1200-1259,1329.0,0.0,0,0,229
3,1,2,20363,9E,N8886A,3283,15249,1524906,TLH,10397,1039707,ATL,1521.0,0.0,1500-1559,1625.0,0.0,0,0,223
4,1,2,20363,9E,N8974C,3284,10397,1039707,ATL,11778,1177801,FSM,1847.0,0.0,1900-1959,1940.0,0.0,0,0,579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565958,31,4,19977,UA,N776UA,200,12016,1201602,GUM,12173,1217305,HNL,749.0,0.0,0700-0759,1832.0,0.0,0,0,3801
565959,31,4,19977,UA,N36280,174,12016,1201602,GUM,14955,1495503,SPN,717.0,0.0,0700-0759,759.0,0.0,0,0,129
565960,31,4,19977,UA,N36280,117,14955,1495503,SPN,12016,1201602,GUM,857.0,0.0,0900-0959,933.0,0.0,0,0,129
565961,31,4,19977,UA,N39726,105,14955,1495503,SPN,12016,1201602,GUM,1820.0,0.0,1800-1859,1854.0,0.0,0,0,129


In [58]:
# Import City Codes to Cities DataFrame
city_code_df = pd.read_csv('../Data/airport_codes_final.csv')
city_code_df

,Airport Code,City
0,ABR,"Aberdeen, SD"
1,ABI,"Abilene, TX"
2,ADK,"Adak Island, AK"
3,CAK,"Akron, OH"
4,ALB,"Albany, NY"
...,...,...
342,OGD,"Ogden, UT"
343,LYH,"Lynchburg, VA"
344,XWA,"Williston, ND"
345,PAE,"Everett, WA"


In [59]:
# Make all cities in lowercase
city_code_lower = city_code_df['City'].str.lower()
city_code_lower

0         aberdeen, sd
1          abilene, tx
2      adak island, ak
3            akron, oh
4           albany, ny
            ...       
342          ogden, ut
343      lynchburg, va
344      williston, nd
345        everett, wa
346         mobile, al
Name: City, Length: 347, dtype: object

In [60]:
# Delete all spaces
city_code_clean = city_code_lower.str.replace(" ", "")
city_code_clean

0        aberdeen,sd
1         abilene,tx
2      adakisland,ak
3           akron,oh
4          albany,ny
           ...      
342         ogden,ut
343     lynchburg,va
344     williston,nd
345       everett,wa
346        mobile,al
Name: City, Length: 347, dtype: object

In [61]:
# Convert the lowercase/no space city series to a dataframe
lower_city_df = city_code_clean.to_frame()
lower_city_df

,City
0,"aberdeen,sd"
1,"abilene,tx"
2,"adakisland,ak"
3,"akron,oh"
4,"albany,ny"
...,...
342,"ogden,ut"
343,"lynchburg,va"
344,"williston,nd"
345,"everett,wa"


In [62]:
# Create a city abreviation dataframe
city_code_abr_df = city_code_df.drop(columns="City")
city_code_abr_df

,Airport Code
0,ABR
1,ABI
2,ADK
3,CAK
4,ALB
...,...
342,OGD
343,LYH
344,XWA
345,PAE


In [63]:
# Inner join the two dataframes to create a clean city code df that can be used by the API
city_code_clean_df = pd.concat([city_code_abr_df, lower_city_df], axis=1, join='inner')
city_code_clean_df

,Airport Code,City
0,ABR,"aberdeen,sd"
1,ABI,"abilene,tx"
2,ADK,"adakisland,ak"
3,CAK,"akron,oh"
4,ALB,"albany,ny"
...,...,...
342,OGD,"ogden,ut"
343,LYH,"lynchburg,va"
344,XWA,"williston,nd"
345,PAE,"everett,wa"


In [64]:
# Return columns that have / 
for city in city_code_clean_df['City']:
    if city.find('/') > -1:
        print(city)


In [65]:
## export the cleaned city list to a csv
city_code_clean_df.to_csv('../../Flight_data_files/city_code_api.csv')

In [66]:
# Create a dataframe for the origin cities
origin_code_df = city_code_clean_df.rename(columns={"Airport Code": "ORIGIN"})

In [67]:
# Create a dataframe for the destination cities
dest_code_df = city_code_clean_df.rename(columns={"Airport Code": "DEST"})
dest_code_df

,DEST,City
0,ABR,"aberdeen,sd"
1,ABI,"abilene,tx"
2,ADK,"adakisland,ak"
3,CAK,"akron,oh"
4,ALB,"albany,ny"
...,...,...
342,OGD,"ogden,ut"
343,LYH,"lynchburg,va"
344,XWA,"williston,nd"
345,PAE,"everett,wa"


In [68]:
flights_2019 = initial_2019[["DAY_OF_MONTH","DAY_OF_WEEK","OP_CARRIER_AIRLINE_ID","OP_CARRIER","TAIL_NUM","OP_CARRIER_FL_NUM","ORIGIN_AIRPORT_ID","ORIGIN_AIRPORT_SEQ_ID","ORIGIN","DEST_AIRPORT_ID","DEST_AIRPORT_SEQ_ID","DEST","DEP_TIME","DEP_DEL15","DEP_TIME_BLK","ARR_TIME","ARR_DEL15","CANCELLED","DIVERTED","DISTANCE"]]
flights_2019

,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,1,2,20363,9E,N8688C,3280,11953,1195302,GNV,10397,1039707,ATL,601.0,0.0,0600-0659,722.0,0.0,0,0,300
1,1,2,20363,9E,N348PQ,3281,13487,1348702,MSP,11193,1119302,CVG,1359.0,0.0,1400-1459,1633.0,0.0,0,0,596
2,1,2,20363,9E,N8896A,3282,11433,1143302,DTW,11193,1119302,CVG,1215.0,0.0,1200-1259,1329.0,0.0,0,0,229
3,1,2,20363,9E,N8886A,3283,15249,1524906,TLH,10397,1039707,ATL,1521.0,0.0,1500-1559,1625.0,0.0,0,0,223
4,1,2,20363,9E,N8974C,3284,10397,1039707,ATL,11778,1177801,FSM,1847.0,0.0,1900-1959,1940.0,0.0,0,0,579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565958,31,4,19977,UA,N776UA,200,12016,1201602,GUM,12173,1217305,HNL,749.0,0.0,0700-0759,1832.0,0.0,0,0,3801
565959,31,4,19977,UA,N36280,174,12016,1201602,GUM,14955,1495503,SPN,717.0,0.0,0700-0759,759.0,0.0,0,0,129
565960,31,4,19977,UA,N36280,117,14955,1495503,SPN,12016,1201602,GUM,857.0,0.0,0900-0959,933.0,0.0,0,0,129
565961,31,4,19977,UA,N39726,105,14955,1495503,SPN,12016,1201602,GUM,1820.0,0.0,1800-1859,1854.0,0.0,0,0,129


In [69]:
# Merge Origin code and the flights dataframes
origin_join_2019_df = pd.merge(origin_code_df,flights_2019, on='ORIGIN', how ='inner')
origin_join_2019_df

,ORIGIN,City,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST_AIRPORT_SEQ_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,ABR,"aberdeen,sd",1,2,20304,OO,N910EV,7363,10141,1014106,...,1348702,MSP,1411.0,1.0,1300-1359,1521.0,1.0,0,0,257
1,ABR,"aberdeen,sd",1,2,20304,OO,N432SW,7365,10141,1014106,...,1348702,MSP,642.0,0.0,0600-0659,800.0,0.0,0,0,257
2,ABR,"aberdeen,sd",2,3,20304,OO,N498CA,7363,10141,1014106,...,1348702,MSP,1310.0,0.0,1300-1359,1424.0,0.0,0,0,257
3,ABR,"aberdeen,sd",2,3,20304,OO,N439SW,7365,10141,1014106,...,1348702,MSP,502.0,0.0,0001-0559,622.0,0.0,0,0,257
4,ABR,"aberdeen,sd",3,4,20304,OO,N8965E,7363,10141,1014106,...,1348702,MSP,1307.0,0.0,1300-1359,1411.0,0.0,0,0,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564543,LYH,"lynchburg,va",29,2,20397,OH,N519AE,5174,13139,1313902,...,1105703,CLT,1108.0,0.0,1100-1159,1223.0,0.0,0,0,175
564544,LYH,"lynchburg,va",30,3,20397,OH,N563NN,5116,13139,1313902,...,1105703,CLT,642.0,0.0,0600-0659,744.0,0.0,0,0,175
564545,LYH,"lynchburg,va",30,3,20397,OH,N712PS,5174,13139,1313902,...,1105703,CLT,1102.0,0.0,1100-1159,1220.0,0.0,0,0,175
564546,LYH,"lynchburg,va",31,4,20397,OH,N709PS,5116,13139,1313902,...,1105703,CLT,648.0,0.0,0600-0659,746.0,0.0,0,0,175


In [70]:
# Change city column to Origin City
origin_join_2019_df = origin_join_2019_df.rename(columns={'City':'ORIGIN_CITY'})
origin_join_2019_df

,ORIGIN,ORIGIN_CITY,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST_AIRPORT_SEQ_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,ABR,"aberdeen,sd",1,2,20304,OO,N910EV,7363,10141,1014106,...,1348702,MSP,1411.0,1.0,1300-1359,1521.0,1.0,0,0,257
1,ABR,"aberdeen,sd",1,2,20304,OO,N432SW,7365,10141,1014106,...,1348702,MSP,642.0,0.0,0600-0659,800.0,0.0,0,0,257
2,ABR,"aberdeen,sd",2,3,20304,OO,N498CA,7363,10141,1014106,...,1348702,MSP,1310.0,0.0,1300-1359,1424.0,0.0,0,0,257
3,ABR,"aberdeen,sd",2,3,20304,OO,N439SW,7365,10141,1014106,...,1348702,MSP,502.0,0.0,0001-0559,622.0,0.0,0,0,257
4,ABR,"aberdeen,sd",3,4,20304,OO,N8965E,7363,10141,1014106,...,1348702,MSP,1307.0,0.0,1300-1359,1411.0,0.0,0,0,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564543,LYH,"lynchburg,va",29,2,20397,OH,N519AE,5174,13139,1313902,...,1105703,CLT,1108.0,0.0,1100-1159,1223.0,0.0,0,0,175
564544,LYH,"lynchburg,va",30,3,20397,OH,N563NN,5116,13139,1313902,...,1105703,CLT,642.0,0.0,0600-0659,744.0,0.0,0,0,175
564545,LYH,"lynchburg,va",30,3,20397,OH,N712PS,5174,13139,1313902,...,1105703,CLT,1102.0,0.0,1100-1159,1220.0,0.0,0,0,175
564546,LYH,"lynchburg,va",31,4,20397,OH,N709PS,5116,13139,1313902,...,1105703,CLT,648.0,0.0,0600-0659,746.0,0.0,0,0,175


In [21]:
# Create a destination dataframe by merging the flights with destination codes
dest_join_2019_df = pd.merge(dest_code_df,flights_2019, on='DEST', how ='inner')
dest_join_2019_df

,DEST,City,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,ABR,"aberdeen,sd",1,2,20304,OO,N910EV,7363,13487,1348702,...,10141,1014106,1220.0,1.0,1100-1159,1343.0,1.0,0,0,257
1,ABR,"aberdeen,sd",1,2,20304,OO,N439SW,7419,13487,1348702,...,10141,1014106,2010.0,0.0,2000-2059,2139.0,0.0,0,0,257
2,ABR,"aberdeen,sd",2,3,20304,OO,N498CA,7363,13487,1348702,...,10141,1014106,1123.0,0.0,1100-1159,1230.0,0.0,0,0,257
3,ABR,"aberdeen,sd",2,3,20304,OO,N935SW,7419,13487,1348702,...,10141,1014106,2226.0,0.0,2200-2259,2342.0,0.0,0,0,257
4,ABR,"aberdeen,sd",3,4,20304,OO,N8965E,7363,13487,1348702,...,10141,1014106,1120.0,0.0,1100-1159,1224.0,0.0,0,0,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564536,LYH,"lynchburg,va",29,2,20397,OH,N519AE,5174,11057,1105703,...,13139,1313902,925.0,0.0,0900-0959,1030.0,0.0,0,0,175
564537,LYH,"lynchburg,va",30,3,20397,OH,N709PS,5123,11057,1105703,...,13139,1313902,2014.0,0.0,2000-2059,2101.0,0.0,0,0,175
564538,LYH,"lynchburg,va",30,3,20397,OH,N712PS,5174,11057,1105703,...,13139,1313902,928.0,0.0,0900-0959,1034.0,0.0,0,0,175
564539,LYH,"lynchburg,va",31,4,20397,OH,N510AE,5123,11057,1105703,...,13139,1313902,2019.0,0.0,2000-2059,2131.0,0.0,0,0,175


In [22]:
# Change city column to Destination City
dest_join_2019_df = dest_join_2019_df.rename(columns={'City':'DEST_CITY'})
dest_join_2019_df

,DEST,DEST_CITY,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,ABR,"aberdeen,sd",1,2,20304,OO,N910EV,7363,13487,1348702,...,10141,1014106,1220.0,1.0,1100-1159,1343.0,1.0,0,0,257
1,ABR,"aberdeen,sd",1,2,20304,OO,N439SW,7419,13487,1348702,...,10141,1014106,2010.0,0.0,2000-2059,2139.0,0.0,0,0,257
2,ABR,"aberdeen,sd",2,3,20304,OO,N498CA,7363,13487,1348702,...,10141,1014106,1123.0,0.0,1100-1159,1230.0,0.0,0,0,257
3,ABR,"aberdeen,sd",2,3,20304,OO,N935SW,7419,13487,1348702,...,10141,1014106,2226.0,0.0,2200-2259,2342.0,0.0,0,0,257
4,ABR,"aberdeen,sd",3,4,20304,OO,N8965E,7363,13487,1348702,...,10141,1014106,1120.0,0.0,1100-1159,1224.0,0.0,0,0,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564536,LYH,"lynchburg,va",29,2,20397,OH,N519AE,5174,11057,1105703,...,13139,1313902,925.0,0.0,0900-0959,1030.0,0.0,0,0,175
564537,LYH,"lynchburg,va",30,3,20397,OH,N709PS,5123,11057,1105703,...,13139,1313902,2014.0,0.0,2000-2059,2101.0,0.0,0,0,175
564538,LYH,"lynchburg,va",30,3,20397,OH,N712PS,5174,11057,1105703,...,13139,1313902,928.0,0.0,0900-0959,1034.0,0.0,0,0,175
564539,LYH,"lynchburg,va",31,4,20397,OH,N510AE,5123,11057,1105703,...,13139,1313902,2019.0,0.0,2000-2059,2131.0,0.0,0,0,175


In [24]:
# Merge the origin and destination dataframes to prepare for the API requests
complete_flight_2019_df = pd.merge(origin_join_2019_df, dest_join_2019_df)
complete_flight_2019_df

,ORIGIN,ORIGIN_CITY,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,DEST_CITY
0,ABR,"aberdeen,sd",1,2,20304,OO,N910EV,7363,10141,1014106,...,MSP,1411.0,1.0,1300-1359,1521.0,1.0,0,0,257,"minneapolis,mn"
1,ABR,"aberdeen,sd",1,2,20304,OO,N432SW,7365,10141,1014106,...,MSP,642.0,0.0,0600-0659,800.0,0.0,0,0,257,"minneapolis,mn"
2,ABR,"aberdeen,sd",2,3,20304,OO,N498CA,7363,10141,1014106,...,MSP,1310.0,0.0,1300-1359,1424.0,0.0,0,0,257,"minneapolis,mn"
3,ABR,"aberdeen,sd",2,3,20304,OO,N439SW,7365,10141,1014106,...,MSP,502.0,0.0,0001-0559,622.0,0.0,0,0,257,"minneapolis,mn"
4,ABR,"aberdeen,sd",3,4,20304,OO,N8965E,7363,10141,1014106,...,MSP,1307.0,0.0,1300-1359,1411.0,0.0,0,0,257,"minneapolis,mn"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563121,LYH,"lynchburg,va",29,2,20397,OH,N519AE,5174,13139,1313902,...,CLT,1108.0,0.0,1100-1159,1223.0,0.0,0,0,175,"charlotte,nc"
563122,LYH,"lynchburg,va",30,3,20397,OH,N563NN,5116,13139,1313902,...,CLT,642.0,0.0,0600-0659,744.0,0.0,0,0,175,"charlotte,nc"
563123,LYH,"lynchburg,va",30,3,20397,OH,N712PS,5174,13139,1313902,...,CLT,1102.0,0.0,1100-1159,1220.0,0.0,0,0,175,"charlotte,nc"
563124,LYH,"lynchburg,va",31,4,20397,OH,N709PS,5116,13139,1313902,...,CLT,648.0,0.0,0600-0659,746.0,0.0,0,0,175,"charlotte,nc"


In [25]:
#Add date format
complete_flight_2019_df['DAY_OF_MONTH'] = '2019-01-' + complete_flight_2019_df['DAY_OF_MONTH'].astype(str)
clean_complete_flight_2019_df = complete_flight_2019_df.rename(columns={"'DAY_OF_MONTH'": "Date"})
clean_complete_flight_2019_df

,ORIGIN,ORIGIN_CITY,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,DEST_CITY
0,ABR,"aberdeen,sd",2019-01-1,2,20304,OO,N910EV,7363,10141,1014106,...,MSP,1411.0,1.0,1300-1359,1521.0,1.0,0,0,257,"minneapolis,mn"
1,ABR,"aberdeen,sd",2019-01-1,2,20304,OO,N432SW,7365,10141,1014106,...,MSP,642.0,0.0,0600-0659,800.0,0.0,0,0,257,"minneapolis,mn"
2,ABR,"aberdeen,sd",2019-01-2,3,20304,OO,N498CA,7363,10141,1014106,...,MSP,1310.0,0.0,1300-1359,1424.0,0.0,0,0,257,"minneapolis,mn"
3,ABR,"aberdeen,sd",2019-01-2,3,20304,OO,N439SW,7365,10141,1014106,...,MSP,502.0,0.0,0001-0559,622.0,0.0,0,0,257,"minneapolis,mn"
4,ABR,"aberdeen,sd",2019-01-3,4,20304,OO,N8965E,7363,10141,1014106,...,MSP,1307.0,0.0,1300-1359,1411.0,0.0,0,0,257,"minneapolis,mn"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563121,LYH,"lynchburg,va",2019-01-29,2,20397,OH,N519AE,5174,13139,1313902,...,CLT,1108.0,0.0,1100-1159,1223.0,0.0,0,0,175,"charlotte,nc"
563122,LYH,"lynchburg,va",2019-01-30,3,20397,OH,N563NN,5116,13139,1313902,...,CLT,642.0,0.0,0600-0659,744.0,0.0,0,0,175,"charlotte,nc"
563123,LYH,"lynchburg,va",2019-01-30,3,20397,OH,N712PS,5174,13139,1313902,...,CLT,1102.0,0.0,1100-1159,1220.0,0.0,0,0,175,"charlotte,nc"
563124,LYH,"lynchburg,va",2019-01-31,4,20397,OH,N709PS,5116,13139,1313902,...,CLT,648.0,0.0,0600-0659,746.0,0.0,0,0,175,"charlotte,nc"


In [26]:
clean_complete_flight_2019_df.to_csv("../../Database/clean_2019_flight_data.csv", index=False)

### 2020 Flights

In [27]:
flights_2020 = initial_2020[["DAY_OF_MONTH","DAY_OF_WEEK","OP_CARRIER_AIRLINE_ID","OP_CARRIER","TAIL_NUM","OP_CARRIER_FL_NUM","ORIGIN_AIRPORT_ID","ORIGIN_AIRPORT_SEQ_ID","ORIGIN","DEST_AIRPORT_ID","DEST_AIRPORT_SEQ_ID","DEST","DEP_TIME","DEP_DEL15","DEP_TIME_BLK","ARR_TIME","ARR_DEL15","CANCELLED","DIVERTED","DISTANCE"]]
flights_2020

,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,1,3,20366,EV,N48901,4397,13930,1393007,ORD,11977,1197705,GRB,1003.0,0.0,1000-1059,1117.0,0.0,0,0,174
1,1,3,20366,EV,N16976,4401,15370,1537002,TUL,13930,1393007,ORD,1027.0,0.0,1000-1059,1216.0,0.0,0,0,585
2,1,3,20366,EV,N12167,4404,11618,1161802,EWR,15412,1541205,TYS,1848.0,0.0,1800-1859,2120.0,0.0,0,0,631
3,1,3,20366,EV,N14902,4405,10781,1078105,BTR,12266,1226603,IAH,1846.0,0.0,1800-1859,2004.0,0.0,0,0,253
4,1,3,20366,EV,N606UX,4407,14524,1452401,RIC,12266,1226603,IAH,1038.0,0.0,1000-1059,1330.0,0.0,0,0,1157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599263,31,5,20363,9E,N331CA,4812,15412,1541205,TYS,11433,1143302,DTW,1002.0,1.0,0700-0759,1128.0,1.0,0,0,443
599264,31,5,20363,9E,N295PQ,4813,11433,1143302,DTW,12478,1247805,JFK,1747.0,0.0,1700-1759,1933.0,0.0,0,0,509
599265,31,5,20363,9E,N294PQ,4814,11996,1199603,GSP,12953,1295304,LGA,554.0,0.0,0600-0659,752.0,0.0,0,0,610
599266,31,5,20363,9E,N228PQ,4815,10397,1039707,ATL,15919,1591904,XNA,1714.0,0.0,1700-1759,1811.0,0.0,0,0,589


In [28]:
# Merge Origin code and the flights dataframes
origin_join_2020_df = pd.merge(origin_code_df,flights_2020, on='ORIGIN', how ='inner')
origin_join_2020_df

,ORIGIN,City,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST_AIRPORT_SEQ_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,ABR,"aberdeen,sd",1,3,20304,OO,N878AS,4244,10141,1014106,...,1348702,MSP,1316.0,0.0,1300-1359,1438.0,0.0,0,0,257
1,ABR,"aberdeen,sd",1,3,20304,OO,N8942A,4255,10141,1014106,...,1348702,MSP,459.0,0.0,0001-0559,605.0,0.0,0,0,257
2,ABR,"aberdeen,sd",2,4,20304,OO,N8965E,4244,10141,1014106,...,1348702,MSP,1417.0,1.0,1300-1359,1518.0,1.0,0,0,257
3,ABR,"aberdeen,sd",2,4,20304,OO,N8933B,4255,10141,1014106,...,1348702,MSP,455.0,0.0,0001-0559,616.0,0.0,0,0,257
4,ABR,"aberdeen,sd",3,5,20304,OO,N477CA,4244,10141,1014106,...,1348702,MSP,1316.0,0.0,1300-1359,1438.0,0.0,0,0,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597708,BFM,"mobile,al",22,3,20436,F9,N939FR,235,10562,1056202,...,1129202,DEN,1218.0,0.0,1200-1259,1418.0,0.0,0,0,1132
597709,BFM,"mobile,al",24,5,20436,F9,N202FR,235,10562,1056202,...,1129202,DEN,1225.0,0.0,1200-1259,1417.0,0.0,0,0,1132
597710,BFM,"mobile,al",27,1,20436,F9,N941FR,235,10562,1056202,...,1129202,DEN,1229.0,0.0,1200-1259,1430.0,0.0,0,0,1132
597711,BFM,"mobile,al",29,3,20436,F9,N941FR,235,10562,1056202,...,1129202,DEN,1234.0,0.0,1200-1259,1445.0,0.0,0,0,1132


In [29]:
# Change city column to Origin City
origin_join_2020_df = origin_join_2020_df.rename(columns={'City':'ORIGIN_CITY'})
origin_join_2020_df

,ORIGIN,ORIGIN_CITY,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST_AIRPORT_SEQ_ID,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,ABR,"aberdeen,sd",1,3,20304,OO,N878AS,4244,10141,1014106,...,1348702,MSP,1316.0,0.0,1300-1359,1438.0,0.0,0,0,257
1,ABR,"aberdeen,sd",1,3,20304,OO,N8942A,4255,10141,1014106,...,1348702,MSP,459.0,0.0,0001-0559,605.0,0.0,0,0,257
2,ABR,"aberdeen,sd",2,4,20304,OO,N8965E,4244,10141,1014106,...,1348702,MSP,1417.0,1.0,1300-1359,1518.0,1.0,0,0,257
3,ABR,"aberdeen,sd",2,4,20304,OO,N8933B,4255,10141,1014106,...,1348702,MSP,455.0,0.0,0001-0559,616.0,0.0,0,0,257
4,ABR,"aberdeen,sd",3,5,20304,OO,N477CA,4244,10141,1014106,...,1348702,MSP,1316.0,0.0,1300-1359,1438.0,0.0,0,0,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597708,BFM,"mobile,al",22,3,20436,F9,N939FR,235,10562,1056202,...,1129202,DEN,1218.0,0.0,1200-1259,1418.0,0.0,0,0,1132
597709,BFM,"mobile,al",24,5,20436,F9,N202FR,235,10562,1056202,...,1129202,DEN,1225.0,0.0,1200-1259,1417.0,0.0,0,0,1132
597710,BFM,"mobile,al",27,1,20436,F9,N941FR,235,10562,1056202,...,1129202,DEN,1229.0,0.0,1200-1259,1430.0,0.0,0,0,1132
597711,BFM,"mobile,al",29,3,20436,F9,N941FR,235,10562,1056202,...,1129202,DEN,1234.0,0.0,1200-1259,1445.0,0.0,0,0,1132


In [30]:
# Create a destination dataframe by merging the flights with destination codes
dest_join_2020_df = pd.merge(dest_code_df,flights_2020, on='DEST', how ='inner')
dest_join_2020_df

,DEST,City,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,ABR,"aberdeen,sd",1,3,20304,OO,N878AS,4244,13487,1348702,...,10141,1014106,1129.0,0.0,1100-1159,1241.0,0.0,0,0,257
1,ABR,"aberdeen,sd",1,3,20304,OO,N8933B,4264,13487,1348702,...,10141,1014106,2013.0,0.0,2000-2059,2125.0,0.0,0,0,257
2,ABR,"aberdeen,sd",2,4,20304,OO,N8965E,4244,13487,1348702,...,10141,1014106,1226.0,1.0,1100-1159,1343.0,1.0,0,0,257
3,ABR,"aberdeen,sd",2,4,20304,OO,N8965E,4264,13487,1348702,...,10141,1014106,2224.0,0.0,2200-2259,2344.0,0.0,0,0,257
4,ABR,"aberdeen,sd",3,5,20304,OO,N477CA,4244,13487,1348702,...,10141,1014106,1136.0,0.0,1100-1159,1247.0,0.0,0,0,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597707,BFM,"mobile,al",22,3,20436,F9,N939FR,234,11292,1129202,...,10562,1056202,754.0,0.0,0800-0859,1132.0,0.0,0,0,1132
597708,BFM,"mobile,al",24,5,20436,F9,N202FR,234,11292,1129202,...,10562,1056202,750.0,0.0,0800-0859,1135.0,0.0,0,0,1132
597709,BFM,"mobile,al",27,1,20436,F9,N941FR,234,11292,1129202,...,10562,1056202,755.0,0.0,0800-0859,1135.0,0.0,0,0,1132
597710,BFM,"mobile,al",29,3,20436,F9,N941FR,234,11292,1129202,...,10562,1056202,750.0,0.0,0800-0859,1207.0,1.0,0,0,1132


In [31]:
# Change city column to Destination City
dest_join_2020_df = dest_join_2020_df.rename(columns={'City':'DEST_CITY'})
dest_join_2020_df

,DEST,DEST_CITY,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE
0,ABR,"aberdeen,sd",1,3,20304,OO,N878AS,4244,13487,1348702,...,10141,1014106,1129.0,0.0,1100-1159,1241.0,0.0,0,0,257
1,ABR,"aberdeen,sd",1,3,20304,OO,N8933B,4264,13487,1348702,...,10141,1014106,2013.0,0.0,2000-2059,2125.0,0.0,0,0,257
2,ABR,"aberdeen,sd",2,4,20304,OO,N8965E,4244,13487,1348702,...,10141,1014106,1226.0,1.0,1100-1159,1343.0,1.0,0,0,257
3,ABR,"aberdeen,sd",2,4,20304,OO,N8965E,4264,13487,1348702,...,10141,1014106,2224.0,0.0,2200-2259,2344.0,0.0,0,0,257
4,ABR,"aberdeen,sd",3,5,20304,OO,N477CA,4244,13487,1348702,...,10141,1014106,1136.0,0.0,1100-1159,1247.0,0.0,0,0,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597707,BFM,"mobile,al",22,3,20436,F9,N939FR,234,11292,1129202,...,10562,1056202,754.0,0.0,0800-0859,1132.0,0.0,0,0,1132
597708,BFM,"mobile,al",24,5,20436,F9,N202FR,234,11292,1129202,...,10562,1056202,750.0,0.0,0800-0859,1135.0,0.0,0,0,1132
597709,BFM,"mobile,al",27,1,20436,F9,N941FR,234,11292,1129202,...,10562,1056202,755.0,0.0,0800-0859,1135.0,0.0,0,0,1132
597710,BFM,"mobile,al",29,3,20436,F9,N941FR,234,11292,1129202,...,10562,1056202,750.0,0.0,0800-0859,1207.0,1.0,0,0,1132


In [32]:
# Merge the origin and destination dataframes to prepare for the API requests
complete_flight_2020_df = pd.merge(origin_join_2020_df, dest_join_2020_df)
complete_flight_2020_df

,ORIGIN,ORIGIN_CITY,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,DEST_CITY
0,ABR,"aberdeen,sd",1,3,20304,OO,N878AS,4244,10141,1014106,...,MSP,1316.0,0.0,1300-1359,1438.0,0.0,0,0,257,"minneapolis,mn"
1,ABR,"aberdeen,sd",1,3,20304,OO,N8942A,4255,10141,1014106,...,MSP,459.0,0.0,0001-0559,605.0,0.0,0,0,257,"minneapolis,mn"
2,ABR,"aberdeen,sd",2,4,20304,OO,N8965E,4244,10141,1014106,...,MSP,1417.0,1.0,1300-1359,1518.0,1.0,0,0,257,"minneapolis,mn"
3,ABR,"aberdeen,sd",2,4,20304,OO,N8933B,4255,10141,1014106,...,MSP,455.0,0.0,0001-0559,616.0,0.0,0,0,257,"minneapolis,mn"
4,ABR,"aberdeen,sd",3,5,20304,OO,N477CA,4244,10141,1014106,...,MSP,1316.0,0.0,1300-1359,1438.0,0.0,0,0,257,"minneapolis,mn"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596152,BFM,"mobile,al",22,3,20436,F9,N939FR,235,10562,1056202,...,DEN,1218.0,0.0,1200-1259,1418.0,0.0,0,0,1132,"denver,co"
596153,BFM,"mobile,al",24,5,20436,F9,N202FR,235,10562,1056202,...,DEN,1225.0,0.0,1200-1259,1417.0,0.0,0,0,1132,"denver,co"
596154,BFM,"mobile,al",27,1,20436,F9,N941FR,235,10562,1056202,...,DEN,1229.0,0.0,1200-1259,1430.0,0.0,0,0,1132,"denver,co"
596155,BFM,"mobile,al",29,3,20436,F9,N941FR,235,10562,1056202,...,DEN,1234.0,0.0,1200-1259,1445.0,0.0,0,0,1132,"denver,co"


In [33]:
#Add date format
complete_flight_2020_df['DAY_OF_MONTH'] = '2020-01-' + complete_flight_2020_df['DAY_OF_MONTH'].astype(str)

In [34]:
# Rename the day of the month column
clean_complete_flight_2020_df = complete_flight_2020_df.rename(columns={"'DAY_OF_MONTH'": "Date"})
clean_complete_flight_2020_df

,ORIGIN,ORIGIN_CITY,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,...,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,DEST_CITY
0,ABR,"aberdeen,sd",2020-01-1,3,20304,OO,N878AS,4244,10141,1014106,...,MSP,1316.0,0.0,1300-1359,1438.0,0.0,0,0,257,"minneapolis,mn"
1,ABR,"aberdeen,sd",2020-01-1,3,20304,OO,N8942A,4255,10141,1014106,...,MSP,459.0,0.0,0001-0559,605.0,0.0,0,0,257,"minneapolis,mn"
2,ABR,"aberdeen,sd",2020-01-2,4,20304,OO,N8965E,4244,10141,1014106,...,MSP,1417.0,1.0,1300-1359,1518.0,1.0,0,0,257,"minneapolis,mn"
3,ABR,"aberdeen,sd",2020-01-2,4,20304,OO,N8933B,4255,10141,1014106,...,MSP,455.0,0.0,0001-0559,616.0,0.0,0,0,257,"minneapolis,mn"
4,ABR,"aberdeen,sd",2020-01-3,5,20304,OO,N477CA,4244,10141,1014106,...,MSP,1316.0,0.0,1300-1359,1438.0,0.0,0,0,257,"minneapolis,mn"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596152,BFM,"mobile,al",2020-01-22,3,20436,F9,N939FR,235,10562,1056202,...,DEN,1218.0,0.0,1200-1259,1418.0,0.0,0,0,1132,"denver,co"
596153,BFM,"mobile,al",2020-01-24,5,20436,F9,N202FR,235,10562,1056202,...,DEN,1225.0,0.0,1200-1259,1417.0,0.0,0,0,1132,"denver,co"
596154,BFM,"mobile,al",2020-01-27,1,20436,F9,N941FR,235,10562,1056202,...,DEN,1229.0,0.0,1200-1259,1430.0,0.0,0,0,1132,"denver,co"
596155,BFM,"mobile,al",2020-01-29,3,20436,F9,N941FR,235,10562,1056202,...,DEN,1234.0,0.0,1200-1259,1445.0,0.0,0,0,1132,"denver,co"


In [35]:
clean_complete_flight_2020_df.to_csv("../../Database/clean_2020_flight_data.csv", index=False)

In [71]:
clean_complete_flight_2020_df.dtypes

ORIGIN                    object
ORIGIN_CITY               object
DAY_OF_MONTH              object
DAY_OF_WEEK                int64
OP_CARRIER_AIRLINE_ID      int64
OP_CARRIER                object
TAIL_NUM                  object
OP_CARRIER_FL_NUM          int64
ORIGIN_AIRPORT_ID          int64
ORIGIN_AIRPORT_SEQ_ID      int64
DEST_AIRPORT_ID            int64
DEST_AIRPORT_SEQ_ID        int64
DEST                      object
DEP_TIME                 float64
DEP_DEL15                float64
DEP_TIME_BLK              object
ARR_TIME                 float64
ARR_DEL15                float64
CANCELLED                  int64
DIVERTED                   int64
DISTANCE                   int64
DEST_CITY                 object
dtype: object